In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HotelID       3889 non-null   int64  
 1   Name Hotel    3889 non-null   object 
 2   Location      3889 non-null   object 
 3   Descriptions  3889 non-null   object 
 4   Rating        3889 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 152.0+ KB


In [4]:
df['Location'].unique()

array(['Phú Quốc', 'TP. Hồ Chí Minh', 'Đà Nẵng', 'Hà Nội', 'Nha Trang',
       'Hội An', 'Đà Lạt', 'Sa Pa', 'Huế', 'Vũng Tàu'], dtype=object)

In [5]:
duplicates = df[df.duplicated(subset="HotelID", keep=False)]
print(duplicates)

      HotelID         Name Hotel         Location  \
18         21         Ruby Hotel        Nha Trang   
19         21         Ruby Hotel          Đà Nẵng   
123       140   The Sun Homestay              Huế   
124       140   The Sun Homestay           Hội An   
391       448        Ohana Hotel           Hà Nội   
392       448        Ohana Hotel  TP. Hồ Chí Minh   
640       736           An Hotel           Hà Nội   
641       736           An Hotel        Nha Trang   
714       824          Mai Hotel           Hà Nội   
715       824          Mai Hotel         Vũng Tàu   
1465     1706     My Dream Hotel            Sa Pa   
1466     1706     My Dream Hotel           Đà Lạt   
1511     1759   Hoang Linh Hotel         Vũng Tàu   
1512     1759   Hoang Linh Hotel          Đà Nẵng   
2608     3033   Phuong Nam Hotel            Sa Pa   
2609     3033   Phuong Nam Hotel         Vũng Tàu   
3140     3632       Prague Hotel  TP. Hồ Chí Minh   
3141     3632       Prague Hotel          Đà N

In [6]:
import re

def get_special_chars(df):
    # Gộp tất cả text trong dataframe (chỉ các cột kiểu object hoặc string)
    text_data = ' '.join(df.select_dtypes(include=['object']).astype(str).values.flatten())
    
    # Tìm tất cả ký tự không phải chữ, số hoặc khoảng trắng
    special_chars = re.findall(r'[^\w\s]', text_data)
    
    # Lấy unique và sắp xếp
    unique_chars = sorted(set(special_chars))
    
    print(f"🔎 Có {len(unique_chars)} ký tự đặc biệt khác nhau trong dữ liệu:\n")
    print(' '.join(unique_chars))
    return unique_chars


In [7]:
specials = get_special_chars(df)

🔎 Có 33 ký tự đặc biệt khác nhau trong dữ liệu:

! " # % & ' ( ) * + , - . / : ; @ [ ] | ~ ̀ ́ ̃ ̉ ̣ ​ – ’ “ ” ☆ ✩


In [8]:
import re
import unicodedata

In [9]:
def clean_vietnamese_text(text):
    if not isinstance(text, str):
        return ""
        
    text = unicodedata.normalize('NFC', text)
    
    text = text.lower()

    text = text.replace('%', ' phần trăm')
    
    text = re.sub(r'/\s*đêm', ' mỗi đêm', text, flags=re.IGNORECASE)
    text = re.sub(r"[^\w\s\-,.]", " ", text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [10]:
text = "xin chàO"
print(clean_vietnamese_text(text))

xin chào


# Embedding với Sentence-BERT

# Load Sentence-BERT

In [11]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# sbert_model = SentenceTransformer('keepitreal/vietnamese-sbert')

# Ghép các trường lại thành 1 text để embedding

In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HotelID       3889 non-null   int64  
 1   Name Hotel    3889 non-null   object 
 2   Location      3889 non-null   object 
 3   Descriptions  3889 non-null   object 
 4   Rating        3889 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 152.0+ KB
None


In [13]:
df.isnull().sum()

HotelID         0
Name Hotel      0
Location        0
Descriptions    0
Rating          0
dtype: int64

In [14]:
df["TextForEmbedding"] = df["Descriptions"].apply(clean_vietnamese_text)

# Sinh embedding cho toàn bộ dataset

In [15]:
embeddings = sbert_model.encode(df["TextForEmbedding"].tolist(), batch_size=32, show_progress_bar=True)
embeddings = normalize(embeddings, axis=1)  # chuẩn hóa theo hàng

Batches:   0%|          | 0/122 [00:00<?, ?it/s]

In [17]:
print("Embedding shape:", embeddings.shape)

Embedding shape: (3889, 768)


In [19]:
print("✅ Embedding done!")
print("Kiểu dữ liệu:", type(embeddings))     # numpy.ndarray
print("Shape:", embeddings.shape)            # (số_samples, số_chiều) ví dụ (1000, 384)
print("Dtype:", embeddings.dtype)            # float32 hoặc float64

✅ Embedding done!
Kiểu dữ liệu: <class 'numpy.ndarray'>
Shape: (3889, 768)
Dtype: float32


In [20]:
norms = np.linalg.norm(embeddings, axis=1)
print(norms[:10])

[1.0000001  1.         0.99999994 1.0000001  1.0000001  1.
 1.         0.99999994 1.         1.        ]


# Lưu vào Milvus

In [21]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility, connections

# Kết nối đến Milvus server

In [22]:
connections.connect(alias="default",host="localhost",port="19530")
collection_name = "hotels_collection_mpnet_base_v2"

# Tạo collection schema

In [23]:
# Kiểm tra xem collection đã tồn tại chưa
if utility.has_collection(collection_name):
    print(f"Collection '{collection_name}' đã tồn tại, xóa và tạo mới lại...")
    utility.drop_collection(collection_name)

# Tạo mới collection
print(f"Tạo mới collection '{collection_name}'...")

fields = [
    FieldSchema(name="HotelID", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="TextForEmbedding", dtype=DataType.FLOAT_VECTOR, dim=768),
    FieldSchema(name="Location", dtype=DataType.VARCHAR, max_length=100), 
    FieldSchema(name="NameHotel", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="Description", dtype=DataType.VARCHAR, max_length=65535),
]

schema = CollectionSchema(fields=fields, description="Hotel dataset embeddings")

collection = Collection(name=collection_name, schema=schema)
print(f"Collection '{collection_name}' đã được tạo mới thành công!")

Tạo mới collection 'hotels_collection_mpnet_base_v2'...
Collection 'hotels_collection_mpnet_base_v2' đã được tạo mới thành công!


# Chuẩn bị dữ liệu để insert

In [24]:
hotel_ids = df["HotelID"].astype(int).tolist()
locations = df["Location"].astype(str).str.strip().tolist()
names = df["Name Hotel"].astype(str).str.strip().tolist()
descriptions = df["Descriptions"].astype(str).str.strip().tolist()

In [25]:
print(type(embeddings))
print(type(hotel_ids))
print(type(locations))
print(type(names))
print(type(descriptions))

<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


# Chunked insert

In [26]:
def chunk_data(data, size):
    for i in range(0, len(data), size):
        yield data[i:i+size]

batch_size = 100

for i, (id_batch, emb_batch, loc_batch, name_batch, des_batch) in enumerate(zip(
    chunk_data(hotel_ids, batch_size),
    chunk_data(embeddings.tolist(), batch_size),
    chunk_data(locations, batch_size),
    chunk_data(names, batch_size),
    chunk_data(descriptions, batch_size)
)):
    collection.insert([
        id_batch,
        emb_batch,
        loc_batch,
        name_batch,
        des_batch
    ])
    print(f"Inserted batch {i+1}")

collection.flush()
print(f"Số lượng bản ghi: {collection.num_entities}")

Inserted batch 1
Inserted batch 2
Inserted batch 3
Inserted batch 4
Inserted batch 5
Inserted batch 6
Inserted batch 7
Inserted batch 8
Inserted batch 9
Inserted batch 10
Inserted batch 11
Inserted batch 12
Inserted batch 13
Inserted batch 14
Inserted batch 15
Inserted batch 16
Inserted batch 17
Inserted batch 18
Inserted batch 19
Inserted batch 20
Inserted batch 21
Inserted batch 22
Inserted batch 23
Inserted batch 24
Inserted batch 25
Inserted batch 26
Inserted batch 27
Inserted batch 28
Inserted batch 29
Inserted batch 30
Inserted batch 31
Inserted batch 32
Inserted batch 33
Inserted batch 34
Inserted batch 35
Inserted batch 36
Inserted batch 37
Inserted batch 38
Inserted batch 39
Số lượng bản ghi: 3889


# Tạo index

In [27]:
index_params = {
    "index_type": "HNSW",
    "metric_type": "COSINE",
    "params": {"M": 8, "efConstruction": 64}
}
# Tạo index cho field vector
collection.release()
collection.drop_index()
collection.create_index(field_name="TextForEmbedding", index_params=index_params)
collection.load()

Status(code=0, message=)